In [ ]:
# todo: check the model cfg: augmentations, policy

In [1]:
import torch
from torch import nn
from utils import parse_cfg
from darknet import Darknet

In [32]:
darknet = Darknet('cfg/yolov3.cfg')

to del, this message should be printed 3ish times
to del, this message should be printed 3ish times
to del, this message should be printed 3ish times
make_layers returns net_info as well. check whether it"s necessary


In [37]:
darknet.layers_info[1]['name']

'convolutional'

In [25]:
t = []
x = torch.randn((2, 3, 608, 608))
t.append(x)
print(t)
x = x + 1
print(t)

[tensor([[[[-0.9665,  0.5351, -0.3072,  ...,  1.1621, -0.1265,  0.6186],
          [ 0.6774,  0.2178, -0.0142,  ...,  0.3335,  0.8363,  2.2494],
          [ 0.6761, -0.6914, -2.3339,  ...,  0.7657, -1.5950, -0.4754],
          ...,
          [-1.5157,  0.0293, -0.2095,  ...,  1.9457, -0.9569,  0.0657],
          [-0.9379,  1.0446, -1.2633,  ..., -0.3359,  0.9698,  0.0881],
          [ 0.5073, -0.1258,  0.0168,  ...,  0.1998,  0.2698, -0.4370]],

         [[ 0.0967,  1.8158, -0.1086,  ...,  1.5871,  1.2152,  0.9816],
          [ 0.0688, -0.7439,  0.3140,  ..., -0.7040, -1.7047,  1.5185],
          [ 1.2309, -0.6477,  2.7795,  ...,  1.1287,  1.0372, -0.8416],
          ...,
          [-2.2436, -1.0690,  0.4535,  ..., -0.5805,  0.0470,  0.3810],
          [-0.8969, -1.4079, -0.5210,  ..., -0.9237,  0.3257,  1.2608],
          [ 0.4565,  1.2727,  0.8348,  ...,  1.6407,  0.1382, -2.1700]],

         [[ 0.7127, -1.0983,  2.0368,  ...,  1.2454,  0.3194, -0.7733],
          [ 0.3774,  0.0611, 

In [26]:
# todo forward use both info and modules 
x = torch.randn((2, 3, 608, 608))

self_layer_list = darknet.layers_list
self_layer_info = darknet.layers_info[1:]

# cache the outputs for route and shortcut layers
outputs = []

for i, layer in enumerate(self_layer_list):
    
    name = self_layer_info[i]['name']
    # TODO: append something after each layer!!!!
    if name in ['convolutional', 'upsample']:
        x = layer(x)
        outputs.append(x)
        
    elif name == 'shortcut':
        # index which is used for shortcut (usually '-3')
        x = outputs[-1] + outputs[layer.frm]
#         x = x + outputs[layer.frm]
        outputs.append(None)
        
    elif name == 'route':
        to_cat = [outputs[route_idx] for route_idx in layer.routes]
        x = torch.cat(to_cat, dim=1)
        outputs.append(None)
        
    elif name == 'yolo':
        # input size: (B, (4+1+classes) * num_achors=255, Gi, Gi)
        B, C, h, w = x.size()
        # read layer's info
        anchors = layer.anchors
        classes = layer.classes
        in_width = layer.in_width
        num_anchs = len(anchors)
        # bbox coords + obj score + class scores
        num_feats = 4 + 1 + classes
        
        x = x.view(B, num_anchs, num_feats, h, w)
        x = x.permute(0, 3, 4, 1, 2).contiguous() # (B, h, w, num_anchs, num_feats)
        x = x.view(1, h*w*num_anchs, num_feats)
        
        try:
            pass
        
        except NameError:
            pass
        
        # output size: (B, \sum_i{Gi*Gi*num_anchors}, (4+1+classes))
        # todo append two times output and detections

SyntaxError: invalid syntax (<ipython-input-26-9ad423323e74>, line 23)

In [7]:
self_layer_list[0]

Sequential(
  (conv_0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn_0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (leaky_0): LeakyReLU(negative_slope=0.1)
)

In [120]:
# TODO proper transformation
a = torch.arange(1*(80+5)*3*5*5).view(1, 255, 5, 5)
# print(a)
print(a[:, :, 0, 1])
a = a.view(1, 3, 85, 5, 5).permute(0, 3, 4, 1, 2).contiguous().view(1, 75, 85)
print(a[:, 5, :])

tensor([[   1,   26,   51,   76,  101,  126,  151,  176,  201,  226,  251,  276,
          301,  326,  351,  376,  401,  426,  451,  476,  501,  526,  551,  576,
          601,  626,  651,  676,  701,  726,  751,  776,  801,  826,  851,  876,
          901,  926,  951,  976, 1001, 1026, 1051, 1076, 1101, 1126, 1151, 1176,
         1201, 1226, 1251, 1276, 1301, 1326, 1351, 1376, 1401, 1426, 1451, 1476,
         1501, 1526, 1551, 1576, 1601, 1626, 1651, 1676, 1701, 1726, 1751, 1776,
         1801, 1826, 1851, 1876, 1901, 1926, 1951, 1976, 2001, 2026, 2051, 2076,
         2101, 2126, 2151, 2176, 2201, 2226, 2251, 2276, 2301, 2326, 2351, 2376,
         2401, 2426, 2451, 2476, 2501, 2526, 2551, 2576, 2601, 2626, 2651, 2676,
         2701, 2726, 2751, 2776, 2801, 2826, 2851, 2876, 2901, 2926, 2951, 2976,
         3001, 3026, 3051, 3076, 3101, 3126, 3151, 3176, 3201, 3226, 3251, 3276,
         3301, 3326, 3351, 3376, 3401, 3426, 3451, 3476, 3501, 3526, 3551, 3576,
         3601, 3626, 3651, 3